In [1]:
import gym
from time import sleep
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import importlib
from stable_baselines3.common.env_checker import check_env

import optical_network_game.game_gym
importlib.reload(optical_network_game.game_gym)
from optical_network_game.game_gym import *

from optical_network_game.requests import *
from optical_network_game.topology_generation import *

import optical_network_game.heuristic
importlib.reload(optical_network_game.heuristic)
from optical_network_game.heuristic import *

from stable_baselines3 import DQN
import pandas as pd

pygame 2.0.3 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Loading RL Model to use

In [2]:
#Insert model_name here based on file name
#model_name =
TL_5_model = DQN.load("DQN_VSNL_TL_5.zip", device='cuda')
TL_10_model = DQN.load("DQN_VSNL_TL_10_ver2.zip", device='cuda')
TL_15_model = DQN.load("DQN_VSNL_TL_15.zip", device='cuda')

In [3]:
#Parameters for results collection
#Holdtime = 10 to 40 (For a traffic load of 5 to 20)
#Number of connection requests = 20
num_req = 20
#request interval = 2 seconds
req_int = 2
#time limit for each connection request = 10 seconds
time_limit = req_int + 1
#bandwidth per link = 5
link_BW = 5

### Obtaining Results<br/>Heuristic Algorithm

In [4]:
#setting up the VSNL topology
nodeList, linkList = createPresetTopology("VSNL", num_slots=link_BW)

#Creating list to store results
results_heuristic = []

#Outer for loop to set hold time from 10 to 40 with increments of 2 to allow for collection of traffic load from 5 to 20 with step of 1
for holdtime in range(10,40):
    
    #number of episodes flag for the inner while loop
    num_episodes = 0

    results_list = {}

    #generating request lists and game environment
    requestList = generateRequests(nodeList, numberOfRequests=num_req, req_interval=req_int, hold_time=holdtime, time_limit=time_limit)
    user = User()
    env = game_gym(nodeList, linkList, requestList, user, dynamic=True)
    eveon = env
    check_env(env)
    heuristic = Heuristic(linkList)
    obs = env.reset()
    
    #for each traffic load setting the game runs for 30 episodes
    while num_episodes < 30:

        action = heuristic.next_action(obs)
        obs, rewards, dones, info = env.step(action)

        if dones == True:
            print(info)

            results_list = info
            results_list['traffic_load'] = holdtime/req_int
            
            #appending the final performance results into the results list for further processing
            results_heuristic.append(info)
            env.reset()
            heuristic = Heuristic(linkList)
            num_episodes += 1
            #debug print
            #print("Episode: " + str(num_episodes))

        env.render()

VSNL Topology Selected
Traffic load is: 5.0
No more requests.
Total reward for this episode is 38298.999999999985
{'bp': 0.0, 'avg_length': 2.9, 'blocked_continuous': 0, 'blocked_contiguous': 0}
No more requests.
Total reward for this episode is 38272.99999999998
{'bp': 0.0, 'avg_length': 2.9, 'blocked_continuous': 0, 'blocked_contiguous': 0}
No more requests.
Total reward for this episode is 38218.999999999985
{'bp': 0.0, 'avg_length': 2.9, 'blocked_continuous': 0, 'blocked_contiguous': 0}
No more requests.
Total reward for this episode is 39383.999999999985
{'bp': 0.0, 'avg_length': 2.8, 'blocked_continuous': 0, 'blocked_contiguous': 0}
No more requests.
Total reward for this episode is 38262.999999999985
{'bp': 0.0, 'avg_length': 2.9, 'blocked_continuous': 0, 'blocked_contiguous': 0}
No more requests.
Total reward for this episode is 38263.99999999998
{'bp': 0.0, 'avg_length': 2.9, 'blocked_continuous': 0, 'blocked_contiguous': 0}
No more requests.
Total reward for this episode is 3

In [5]:
#after for loop is completed, save the results list as a csv file
df_heuristic = pd.DataFrame(results_heuristic)
df_heuristic
# saving the dataframe 
#INSERT PROPER RESULTS NAME HERE
df_heuristic.to_csv('results_heuristic.csv')

### RL Model Results

## Traffic Load 5

In [6]:
#setting up the VSNL topology
nodeList, linkList = createPresetTopology("VSNL", num_slots=link_BW)

results_RL_TL_5 = []

#Outer for loop to set hold time from 10 to 40 with increments of 2 to allow for collection of traffic load from 5 to 20 with step of 1
for holdtime in range(10,40):
    
    #number of episodes flag for the inner while loop
    num_episodes = 0

    results_list = {}
    
    #generating request lists and game environment
    requestList = generateRequests(nodeList, numberOfRequests=num_req, req_interval=req_int, hold_time=holdtime, time_limit=time_limit)
    user = User()
    env = game_gym(nodeList, linkList, requestList, user, dynamic=True)
    eveon = env
    check_env(env)
    heuristic = Heuristic(linkList)
    obs = env.reset()
    
    #for each traffic load setting the game runs for 30 episodes
    while num_episodes < 30:

        action, states_ = TL_5_model.predict(obs, deterministic=True )
        obs, rewards, dones, info = env.step(action)

        if dones == True:
            #print(info)

            results_list = info
            results_list['traffic_load'] = holdtime/req_int
            
            #appending the final performance results into the results list for further processing
            results_RL_TL_5.append(info)
            env.reset()
            heuristic = Heuristic(linkList)
            num_episodes += 1
            #debug print
            #print("Episode: " + str(num_episodes))

        env.render()

VSNL Topology Selected
Traffic load is: 5.0
Stop hitting yourself
No more requests.
Total reward for this episode is -16355.000000000002
Stop hitting yourself
Stop hitting yourself
No more requests.
Total reward for this episode is 13132.0
Stop hitting yourself
No more requests.
Total reward for this episode is 13916.999999999996
No more requests.
Total reward for this episode is 24524.0
Stop hitting yourself
No more requests.
Total reward for this episode is 14167.000000000007
Stop hitting yourself
No more requests.
Total reward for this episode is 2620.0000000000045
Stop hitting yourself
Stop hitting yourself
No more requests.
Total reward for this episode is -12334.0
Stop hitting yourself
No more requests.
Total reward for this episode is 10825.0
Stop hitting yourself
No more requests.
Total reward for this episode is 25418.0
Stop hitting yourself
Stop hitting yourself
No more requests.
Total reward for this episode is 14163.999999999998
Stop hitting yourself
Stop hitting yourself
N

In [7]:
#after for loop is completed, save the results list as a csv file
df_TL_5 = pd.DataFrame(results_RL_TL_5)
# saving the dataframe 
#INSERT PROPER RESULTS NAME HERE
df_TL_5.to_csv('results_RL_TL_5.csv')

## Traffic Load 10

In [8]:
#setting up the VSNL topology
nodeList, linkList = createPresetTopology("VSNL", num_slots=link_BW)

results_RL_TL_10 = []

#Outer for loop to set hold time from 10 to 40 with increments of 2 to allow for collection of traffic load from 5 to 20 with step of 1
for holdtime in range(10,40):
    
    #number of episodes flag for the inner while loop
    num_episodes = 0

    results_list = {}
    
    #generating request lists and game environment
    requestList = generateRequests(nodeList, numberOfRequests=num_req, req_interval=req_int, hold_time=holdtime, time_limit=time_limit)
    user = User()
    env = game_gym(nodeList, linkList, requestList, user, dynamic=True)
    eveon = env
    check_env(env)
    heuristic = Heuristic(linkList)
    obs = env.reset()
    
    #for each traffic load setting the game runs for 30 episodes
    while num_episodes < 30:

        action, states_ = TL_10_model.predict(obs, deterministic=True )
        obs, rewards, dones, info = env.step(action)

        if dones == True:
            #print(info)

            results_list = info
            results_list['traffic_load'] = holdtime/req_int
            
            #appending the final performance results into the results list for further processing
            results_RL_TL_10.append(info)
            env.reset()
            heuristic = Heuristic(linkList)
            num_episodes += 1
            #debug print
            #print("Episode: " + str(num_episodes))

        env.render()

VSNL Topology Selected
Traffic load is: 5.0
No more requests.
Total reward for this episode is 20031.0
No more requests.
Total reward for this episode is 13091.000000000002
No more requests.
Total reward for this episode is 10038.000000000004
No more requests.
Total reward for this episode is 11805.000000000002
Stop hitting yourself
No more requests.
Total reward for this episode is 25588.00000000001
Stop hitting yourself
Stop hitting yourself
No more requests.
Total reward for this episode is 11057.000000000002
Too many invalid actions.
Total reward for this episode is -176.99999999999534
No more requests.
Total reward for this episode is 24625.000000000007
Too many invalid actions.
Total reward for this episode is -195.9999999999951
No more requests.
Total reward for this episode is 24597.00000000001
No more requests.
Total reward for this episode is 11591.999999999998
Stop hitting yourself
No more requests.
Total reward for this episode is 1491.0000000000055
No more requests.
Total 

In [9]:
#after for loop is completed, save the results list as a csv file
df_TL_10 = pd.DataFrame(results_RL_TL_10)
# saving the dataframe 
#INSERT PROPER RESULTS NAME HERE
df_TL_10.to_csv('results_RL_TL_10.csv')

## Traffic Load 15

In [10]:
#setting up the VSNL topology
nodeList, linkList = createPresetTopology("VSNL", num_slots=link_BW)

results_RL_TL_15 = []

#Outer for loop to set hold time from 10 to 40 with increments of 2 to allow for collection of traffic load from 5 to 20 with step of 1
for holdtime in range(10,40):
    
    #number of episodes flag for the inner while loop
    num_episodes = 0

    results_list = {}
    
    #generating request lists and game environment
    requestList = generateRequests(nodeList, numberOfRequests=num_req, req_interval=req_int, hold_time=holdtime, time_limit=time_limit)
    user = User()
    env = game_gym(nodeList, linkList, requestList, user, dynamic=True)
    eveon = env
    check_env(env)
    heuristic = Heuristic(linkList)
    obs = env.reset()
    
    #for each traffic load setting the game runs for 30 episodes
    while num_episodes < 30:

        action, states_ = TL_15_model.predict(obs, deterministic=True )
        obs, rewards, dones, info = env.step(action)

        if dones == True:
            #print(info)

            results_list = info
            results_list['traffic_load'] = holdtime/req_int
            
            #appending the final performance results into the results list for further processing
            results_RL_TL_15.append(info)
            env.reset()
            heuristic = Heuristic(linkList)
            num_episodes += 1
            #debug print
            #print("Episode: " + str(num_episodes))

        env.render()

VSNL Topology Selected
Traffic load is: 5.0
No more requests.
Total reward for this episode is 9642.000000000004
Too many invalid actions.
Total reward for this episode is -563.9999999999958
No more requests.
Total reward for this episode is -428.0000000000001
Too many invalid actions.
Total reward for this episode is 9861.000000000007
Stop hitting yourself
No more requests.
Total reward for this episode is 7859.0
No more requests.
Total reward for this episode is 13103.000000000004
No more requests.
Total reward for this episode is 11997.000000000005
No more requests.
Total reward for this episode is -361.9999999999962
No more requests.
Total reward for this episode is 25337.0
Stop hitting yourself
No more requests.
Total reward for this episode is 331.0000000000059
No more requests.
Total reward for this episode is 11824.0
No more requests.
Total reward for this episode is 13096.0
No more requests.
Total reward for this episode is 23179.000000000004
No more requests.
Total reward for

In [11]:
#after for loop is completed, save the results list as a csv file
df_TL_15 = pd.DataFrame(results_RL_TL_15)
# saving the dataframe 
#INSERT PROPER RESULTS NAME HERE
df_TL_15.to_csv('results_RL_TL_15.csv')